# ΜΑΡΙΑ ΚΑΙΚΤΖΟΓΛΟΥ - 03400052
# ΔΗΜΗΤΡΗΣ ΖΕΡΚΕΛΙΔΗΣ - 03400049

# ΜΕΡΟΣ 2
## Χρήση σημασιολογικών αναπαραστάσεων για ανάλυση συναισθήματος

### Βήμα 16: Δεδομένα και προεπεξεργασία

__(a)__ Κατεβάζουμε τα δεδομένα που μας ζητάει στο ερώτημα από το 

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

__(b)__ Προεπεξεργασία δεδομένων

Χρησιμοποιούμε τον κώδικα από το github, για το δίαβασμα των δεδομένων και κάνουμε μια αλλαγή στη συνάρτηση tokenize, ώστε να διαβάζει μια λίστα απο προτάσεις και όχι μια λίστα ξεχωριστά για να μας επιστρέφει tokenized sentences.

In [1]:
import os

data_dir = './aclImdb/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
pos_train_dir = os.path.join(train_dir, 'pos')
neg_train_dir = os.path.join(train_dir, 'neg')
pos_test_dir = os.path.join(test_dir, 'pos')
neg_test_dir = os.path.join(test_dir, 'neg')

# For memory limitations. These parameters fit in 8GB of RAM.
# If you have 16G of RAM you can experiment with the full dataset / W2V

MAX_NUM_SAMPLES = 5000 
# Load first 1M word embeddings. This works because GoogleNews are roughly
# sorted from most frequent to least frequent.
# It may yield much worse results for other embeddings corpora
NUM_W2V_TO_LOAD = 100000 # το μείωσαμε λόγω ram

import numpy as np

SEED = 42

# Fix numpy random seed for reproducibility
np.random.seed(42)

try:
    import glob2 as glob
except ImportError:
    import glob

import re

def strip_punctuation(s):
    return re.sub(r'[^a-zA-Z\s]', ' ', s)

def preprocess(s):
    return re.sub('\s+',' ', strip_punctuation(s).lower())

def tokenize(s):
    array =[]
    for i in s:
        tmp = i.split(' ')
        tmp_arr = [x for x in tmp]
        array.append(tmp_arr)
    return array

def preproc_tok(s):
    return tokenize(preprocess(s))

def read_samples(folder, preprocess=lambda x: x):
    samples = glob.iglob(os.path.join(folder, '*.txt'))
    data = []
    for i, sample in enumerate(samples):
        if MAX_NUM_SAMPLES > 0 and i == MAX_NUM_SAMPLES:
            break
        with open(sample, 'r') as fd:
            x = [preprocess(l) for l in fd][0]
            data.append(x)
    return data

def create_corpus(pos, neg):
    corpus = np.array(pos + neg) # enwnw tis 2 listes 
    y = np.array([1 for _ in pos] + [0 for _ in neg]) # ftiaxnw lista y me 1 sta pos kai 0 sta neg
    indices = np.arange(y.shape[0]) # anakatevw ta dedomena mou gia kalitero training
    np.random.shuffle(indices)
    return list(corpus[indices]), list(y[indices]) # epistrefw lista me 2 ipolistes , mia gia to review,mia gia to label 1 i 0

In [2]:
positive_train = read_samples(pos_train_dir,preprocess )
positive_test = read_samples(pos_test_dir,preprocess )

In [3]:
negative_train = read_samples(neg_train_dir,preprocess )
negative_test  = read_samples(neg_test_dir,preprocess )

Παρακάτω βλέπουμε τη δομή των παραπάνω λιστών που φτιάξαμε. Είναι όπως φαίνεται το κάθε entry της λίστας ένα string που αντιπροσοπεύει τα reviews.

In [4]:
positive_train[:2]

['i was lucky enough to catch this movie while volunteering at the maryland film festival i ve always been a fan of classic horror films and especially the gimmicks of william castle so this was definitely a must see for me br br this is about the life and work of william castle who in my opinion was an underrated director true he made some cheap budget schlocky horror films but he added something to these films real live theater gimmicks that you don t see anymore for example he had nurses in case someone had a heart attack at his movies and put vibrators at the bottoms of chairs in the tingler br br this is truly a well made documentary and brings this rather shadowed director into the light and celebrated his contributions to horror cinema it also paints castle as a larger than life character who was very well liked and had a smile on his face br br unlike most film documentaries that mostly show testaments from film historians spine tingler shows interviews mostly from his family m

Αλλάξαμε λίγο την tokenize , ώστε να επεξεργάζεται λίστα με reviews και επιστρέφει μια tokenized λίστα ανά πρόταση.

Δηλαδή έχουμε μια λίστα με υπολίστες tokenized sentences.

In [5]:
tokenized_pos_train = tokenize(positive_train)
tokenized_pos_test  = tokenize(positive_test)
tokenized_neg_train = tokenize(negative_train)
tokenized_neg_test  = tokenize(negative_test)

Παρακάτω θα δείξουμε ένα παράδειγμα μιας tokenized πρότασης.

In [6]:
tokenized_neg_test[:2]

[['',
  'out',
  'of',
  'br',
  'br',
  'this',
  'film',
  'was',
  'neither',
  'funny',
  'as',
  'a',
  'whole',
  'nor',
  'was',
  'it',
  'even',
  'worthing',
  'investing',
  'any',
  'kind',
  'of',
  'emotion',
  'into',
  'the',
  'characters',
  'eugene',
  'levy',
  'is',
  'probably',
  'the',
  'most',
  'funny',
  'the',
  'rest',
  'of',
  'the',
  'cast',
  'do',
  'their',
  'jobs',
  'but',
  'the',
  'story',
  'never',
  'really',
  'gets',
  'very',
  'deep',
  'and',
  'there',
  'are',
  'a',
  'lot',
  'of',
  'holes',
  'in',
  'the',
  'plot',
  'that',
  'never',
  'get',
  'filled',
  'this',
  'just',
  'wasn',
  't',
  'very',
  'much',
  'fun',
  'despite',
  'being',
  'funny',
  'at',
  'times',
  'br',
  'br',
  ''],
 ['although',
  'coming',
  'after',
  'three',
  'star',
  'wars',
  'krull',
  'countless',
  'others',
  'this',
  'movie',
  'would',
  'look',
  'outdated',
  'in',
  'the',
  's',
  'sfx',
  'mostly',
  'consist',
  'of',
  's',


To παραπάνω output είναι οι 2 πρώτες αρνητικές κριτικές του test set σε tokenized προτάσεις.

Κατασκευή Corpus στο οποίο ενώνουμε τα training data που είναι διαχωρισμένα σε negative , positive και φτιάχνουμε μια επιπλέον λίστα με τα αντίστοιχα labels 1 για positive  , 0 για negative.

Στη συνέχεια επιστρέφουμε μια ανακατεμένη λίστα με 2 υπολίστες που περιέχουν παράλληλα και οι 2 τα δεδομένα reviews - labels.

In [7]:
corpus = create_corpus(positive_train, negative_train)

Παρακάτω τυπώνουμε 2 κριτικές που είναι στην 1η υπολίστα του corpus.

In [7]:
corpus[0][:2]

['she has been catapulted from to with magic dust involved courtesy the year old matt but nothing is made of that except as an unexplained device new york city especially central park but also every other slice of the place incorporated into the movie seems hope filled and easily livable and save for lucy there s no villain in jenna s adult life and even lucy is not cast as monstrous only as a nasty year old grown years more devious chris the one time boy object of jenna s yearning is now a porky cab driver and you have seen enough films to know that matt will play a major role in jenna s future you don t know quite what might impede this before it is finally achieved though i m here to whisper in your ear so to speak that the device is not unique in fact not only is this a variation on the theme of tom hanks big though nowhere near as fine it is also a strictly by the book version of this subset of the cinderella story ',
 'this movie is a journey through the mind of a screenwriter ca

Τώρα θα τυπώσουμε και τα 2 labels αυτών των κριτικών στη 2η υπολίστα της λίστας corpus.

In [8]:
corpus[1][:2]

[0, 1]

Όπως φαίνεται και οι 2 κριτικές είναι αρνητικές.

### Βήμα 17: Κατασκευή ΒΟW αναπαραστάσεων και ταξινόμηση

__(a)__

Στην περίπτωση, που έχουμε bag of words matrix , σημαίνει ότι κάθε γραμμή  περιέχει το vocabulary μας και κάθε στήλη περιέχει ένα document. Δηλαδή έχουμε ένα πίνακα |v| x |d|. 

Ώστοσο, αν μείνουμε στο πόσες φορές απλά μια λέξη εμφανίζεται σε ένα document δεν βγάζουμε καλά συμπεράσματα εξαιτίας λέξεων που χρησιμοποιούντε συχνά όπως το the, it , they. Ουσιαστικά, μας ενδιαφέρουν λέξεις κοντά σε άλλες με μεγάλη συχνότητα αλλά όχι και με πολύ μεγάλη συχνότητα όπως οι λέξεις που αναφέραμε. 

Αυτό έρχεται να κάνει balance o αλγόριθμος tf-idf. Τα βάρη tf-idf είναι ένα γινόμενο 2 όρων, του term frequency και του inverse document frequency.

O 1ος όρος δίνει βαρύτητα στις λέξεις που εμφανίζονται σε ένα κείμενο συχνά , ενώ ο 2ος όρος δίνει βαρύτητα σε λέξεις που είναι πιο πολύ σπάνιες, για παράδειγμα μπορεί να εμφανίζονται μόνο σε 1 κείμενο από τα 100 , ενώ άλλες λέξεις εμφανίζονται και στα 100 κείμενα. 

__(b)__

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [13]:
import pickle
X = vectorizer.fit_transform(corpus[0])
pickle.dump(vectorizer.vocabulary_,open("feature.pkl","wb"))

__(c)__

Aρχικά κάνουμε train το model με τα δεδομένα για εκπαίδευση που έχουμε.

In [24]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs',max_iter=1000)
y = corpus[1]
clf.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Τώρα θα υπολογίσουμε το αccuracy στα test δεδομένα. Αρχικά φτίαχνω ένα corpus_test όπως πριν όμως για τα δεδομένα για πρόγνωση.

In [25]:
corpus_test = create_corpus(positive_test, negative_test)

In [26]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
X_test = loaded_vec.fit_transform(corpus_test[0])
y_test = corpus_test[1]

In [27]:
clf.score(X_test,y_test)

0.8579

Όπως βλέπουμε έχουμε αρκετά υψηλό accuracy.

Ο λόγος που χρησιμοποιήσαμε το module pickle ήταν για να αποθηκεύσουμε το εκπαιδευμένο μοντέλο σε ένα αρχείο ώστε μετά να κάνει transform το test set βάσει αυτού του μοντέλου και όχι άλλου, ώστε να έχουμε τα ίδια χαρακτηριστικά.

__(d)__

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True) #smooth_idf=True helps in division by 0
tdidf_xtrain = transformer.fit_transform(X)
tdidf_xtest  = transformer.fit_transform(X_test)

In [18]:
clf_tdf = LogisticRegression(solver='lbfgs',max_iter=1000)
clf_tdf.fit(tdidf_xtrain,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
clf_tdf.score(tdidf_xtest,y_test)

0.8684

Παρατηρούμε πως υπάρχει βελτίωση στο SCORE με τη χρήση του αλγορίθμου td-idf.

### Βήμα 18: Χρήση Word2Vec αναπαραστάσεων για ταξινόμηση.

Παρακάτω ακολουθεί κώδικας από το βήμα 9 της προπαρασκευαστικής για να πάρουμε τα κατάλληλα δεδομένα ξανά.


In [16]:
path_sherlock = '/home/zerkes/Desktop/DSML - MASTER/1st semester/NLP/lab1/προπαρασκευαστικό/1book.txt' # path for the book!

import re
def tokenize(s):
    s = s.strip().lower() # removing spaces in beginning and ending of a sentence + lowering the letters
    s = re.sub(r'\W+', ' ', s) # '\W == [^a-zA-Z0-9_] keeping only alphanumerical values
    s = s.replace('\n', ' ')
    s = s.split()
    return s

def corpus_sentence(path):
    array=[]
    file = open(path_sherlock)
    lines = [line for line in file.readlines() if line.strip()] #keeping non empty lines
    file.close()
    for l in lines:
        l = l.strip().lower()
        l = re.sub(r'\W+', ' ', l)
        l = l.replace('\n', ' ')
        array.append(l)
    return array   

def corpus_sentence_tokenize(path):
    array=[]
    file = open(path_sherlock)
    lines = [line for line in file.readlines() if line.strip()] #keeping non empty lines
    file.close()
    for l in lines:
        array.append(tokenize(l))
    return array   

In [17]:
corpus_sentence_tkn = corpus_sentence_tokenize(path_sherlock) # tokenized sentences
from gensim.models import Word2Vec
model = Word2Vec(sg=1,min_count=1,size=100,window=5)
model.build_vocab(corpus_sentence_tkn)

In [18]:
model.train(corpus_sentence_tkn,total_examples=model.corpus_count, epochs=1000, compute_loss=True)

(78007136, 109226000)

__(a)__
θα δούμε από το corpus του train πόσες λέξεις έχουμε OOV από το word2vec του ερωτήματος 9, ενώ ταυτόχρονα θα υπολογίσουμε και την κάθε πρόταση ως αναπαράσταση word2vec.

### oov ποσοστό

In [61]:
index2word_set = set(model.wv.index2word)
oov_counter = 0
n_words = 0
for i in range(len(corpus[0])):
    words = corpus[0][i].split()
    for word in words:
        if word not in index2word_set:
            oov_counter +=1
        n_words +=1
print((oov_counter/n_words)*100)

18.70524690516528


Παρατηρούμε ότι έχουμε ένα ποσοστό 18% λέξεω εκτός λεξιλογίου στο μοντέλο μας. Κάτι που είναι λογικό καθώς έχουμε πολύ μικρό dataset.

### Yπολογισμός προτάσεων ως word2vec

In [19]:
import numpy as np
from scipy import spatial

index2word_set = set(model.wv.index2word)

def calculate_sentences(sentence, model,index2word_set,size=100):
    words = sentence.split()
    feature_vec = np.zeros((size, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
        else:
            #word is OOV
            feature_vec = np.add(feature_vec, np.zeros((size, ), dtype='float32'))
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [20]:
index2word_set = set(model.wv.index2word)

In [33]:
#corpus[i][0].split()
sentences2vec_train=[]
for i in range(len(corpus[0])):
    sentences2vec_train.append(calculate_sentences(corpus[0][i], model, index2word_set))

/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [34]:
sentences2vec_test=[]
for i in range(len(corpus_test[0])):
    sentences2vec_test.append(calculate_sentences(corpus_test[0][i], model, index2word_set))

/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


__(β)__ Κάνουμε training με logistic regression και έπειτα βλέπουμε τα αποτελέσματα στο test set.

In [66]:
from sklearn.linear_model import LogisticRegression

#logistic regression fit
clf3 = LogisticRegression(solver='lbfgs',max_iter=1000)
clf3.fit(sentences2vec_train,corpus[1])
#predict
print(clf3.score(sentences2vec_test,corpus_test[1]))

0.6958


Παρατηρούμε πως τα ποσοστά είναι αρκετά χαμηλά , γύρω στο 70%. Αυτό είναι λογικό καθώς στα δεδομένα μας, έχουμε αρκετές λέξεις που λείπουν από από τα δεδομένα που κάνουμε predict, ενώ επίσης έχουμε μικρό dataset για τον υπολογισμό word2vec διανυσμάτων, με αποτέλεσμα τα διανύσματα με κοντινές αποστάσεις, οι αντίστοιχες λέξεις τους να μην είναι κοντά σημασιολογικά.

__(γ)__ , __(δ)__

H μεταβλήτη NUM_W2V_TO_LOAD έχει μειωθεί στο 100 000 λόγω έλλειψης μνήμης.

In [21]:
from gensim.models import KeyedVectors
model_google = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True, limit=NUM_W2V_TO_LOAD)

In [49]:
#βάζουμε σε μια λίστα τις λέξεις που χρησιμοποιήσαμε στο ερώτημα 9
random_words = ['vestige','allude','fulfill','jersey','restraint','property','assume','method','chuckled','fashioned']

In [70]:
for i in random_words:
    print("Most Similar words for: ",i)
    print(model_google.most_similar(positive=i))
    print("\n")

Most Similar words for:  vestige
[('vestiges', 0.7638673186302185), ('remnant', 0.6451860666275024), ('relic', 0.6033318638801575), ('anachronism', 0.5288219451904297), ('bastion', 0.5030342936515808), ('bastions', 0.4654688835144043), ('bygone_era', 0.45959675312042236), ('remnants', 0.45440924167633057), ('anachronistic', 0.44333261251449585), ('throwback', 0.42959529161453247)]


Most Similar words for:  allude
[('alluded', 0.5936486124992371), ('alludes', 0.5497958660125732), ('imply', 0.5486440658569336), ('mention', 0.5430586338043213), ('depict', 0.5281932353973389), ('cite', 0.5100091099739075), ('foreshadow', 0.508596658706665), ('alluding', 0.5042288303375244), ('expound', 0.5022125840187073), ('describe', 0.49938517808914185)]


Most Similar words for:  fulfill
[('fulfilled', 0.7905531525611877), ('fulfilling', 0.784023642539978), ('fulfills', 0.7497856616973877), ('satisfy', 0.6402673721313477), ('meet', 0.5674575567245483), ('fulfillment', 0.5567388534545898), ('unfulfille

Παρατηρούμε πως με αυτό το πολύ μεγαλύτερο dataset , έχουμε πολύ καλύτερα αποτελέσματα και οι λέξεις είναι αρκετά κοντά νοηματικά.

__(ε)__ Τώρα θα φτιάξουμε ένα logistic regression μοντέλο με neural bag of words αναπαραστάσεις , με βάση το word2vec της google.

In [71]:
index_google = set(model_google.wv.index2word)
sentences2google_train=[]
for i in range(len(corpus[0])):
    sentences2google_train.append(calculate_sentences(corpus[0][i], model_google, index_google,size=300))

sentences2google_test=[]
for i in range(len(corpus_test[0])):
    sentences2google_test.append(calculate_sentences(corpus_test[0][i], model_google, index_google,size=300))

/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [72]:
#logistic regression fit
clf4 = LogisticRegression(solver='lbfgs',max_iter=1000)
clf4.fit(sentences2google_train,corpus[1])
#predict
print(clf4.score(sentences2google_test,corpus_test[1]))

0.8361


Το ποσοστό όπως φαίνεται αυξήθηκε κατά 14%. Γεγονός, που σημαίνει πως όσο περισσότερα δεδομένα έχεις τόσο καλύτερα μπορείς να αναπαραστήσεις τις προτάσεις σου και να φτάσεις σε καλύτερες προγνώσεις.

__(στ)__

Τώρα θα δημιουργήσουμε αναπαραστάσεις όπως πριν αλλά με το συνδυασμό του td-idf. Αρχικά, το δοκιμάσουμε στο δικό μας μοντέλο και έπειτα στο επόμενω ερώτημα στης google.

To td-idf θα το εκπαιδεύσουμε πάνω στο στο corpus των reviews του imdb στο training set, γιατί έχει πολλά documents.

Στο scikit παίρνουμε τα score tdidf με τη χρήση του attribute _tfidf.idf_ 


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(use_idf=True)
vectorizer.fit_transform(corpus[0])


feature_names = vectorizer.get_feature_names()
idf = vectorizer._tfidf.idf_

tdidf_dict = dict(zip(feature_names, idf)) # key: words , value: td-idf score

def calculate_sentences_td_idf(sentence, model,index2word_set,size=100,tdidf_dict=tdidf_dict):
    words = sentence.split()
    feature_vec = np.zeros((size, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            if word in tdidf_dict.keys():
                feature_vec = np.add(feature_vec , model[word] * tdidf_dict[word])
            else:
                feature_vec = np.add(feature_vec, model[word])
        else:
            #word is OOV
            feature_vec = np.add(feature_vec, np.zeros((size, ), dtype='float32'))
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

Tώρα θα μετατρέψουμε με το word2vec το δικό μας τις προτάσεις σε NBOW ξανά αλλά με τη χρήση των td-idf score και θα δούμε αν έχει διαφορά στο αποτέλεσμα.

In [29]:
#corpus[i][0].split()
sentences2vec_tdidf_train=[]
for i in range(len(corpus[0])):
    sentences2vec_tdidf_train.append(calculate_sentences_td_idf(corpus[0][i], model, index2word_set))

sentences2vec_tdidf_test=[]
for i in range(len(corpus_test[0])):
    sentences2vec_tdidf_test.append(calculate_sentences_td_idf(corpus_test[0][i], model, index2word_set))

#logistic regression fit
clf4 = LogisticRegression(solver='lbfgs',max_iter=1000)
clf4.fit(sentences2vec_tdidf_train,corpus[1])
#predict
print(clf4.score(sentences2vec_tdidf_test,corpus_test[1]))

/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


0.6728


παρατηρούμε ότι δεν υπήρχε σημαντική διαφορά στο αποτέλεσμα. Έχουμε μείωση του ποσοστού από 0.6958 που ήταν.

__(ζ)__

Σε αυτό το ερώτημα θα επαναλάβουμε τη διαδικασία του προηγούμενου ερωτήματος , αλλά στο word2vec της google.

In [30]:
index_google = set(model_google.wv.index2word)
sentences2google_train=[]
for i in range(len(corpus[0])):
    sentences2google_train.append(calculate_sentences_td_idf(corpus[0][i], model_google, index_google,size=300))

sentences2google_test=[]
for i in range(len(corpus_test[0])):
    sentences2google_test.append(calculate_sentences_td_idf(corpus_test[0][i], model_google, index_google,size=300))

#logistic regression fit
clf5 = LogisticRegression(solver='lbfgs',max_iter=1000)
clf5.fit(sentences2google_train,corpus[1])
#predict
print(clf5.score(sentences2google_test,corpus_test[1]))

/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


0.8432


Στο συγκεκριμένο μοντέλο παρατηρούμε μια μικρή άυξηση του ποσοστού της τάξεως του 1%.
Γεγονός, που σημαίνει ότι η χρήση των βαρών td-idf ταιριάζει καλύτερα στο word2vec της google, ίσως γιατί έχει δημιουργήσει το μοντέλο της πάνω σε ένα τεράστιο όγκο δεδομένων.

### Βήμα 19 : Extra Credit

__(a)__ Αρχικά θα πειραματιστούμε με άλλους ταξινομητές όπως KNN , SVM για να δούμε τι ποσοστά θα μας δώσουνε.

__KNN__

Mε το word2vec της google:

In [31]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(sentences2google_train, corpus[1])
print(neigh.score(sentences2google_test,corpus_test[1]))

0.7529


Mε το δικό μας word2vec:

In [35]:
neigh2 = KNeighborsClassifier(n_neighbors=5)
neigh2.fit(sentences2vec_train, corpus[1])
print(neigh2.score(sentences2vec_test,corpus_test[1]))

0.6215


__SVM__

Oμοίως με τη χρήση αλγορίθμου SVM για το μοντέλο της google. Κάνουμε χρήση του kernel rbf καθώς έχουμε κλάσεις που δεν είναι γραμμικά διαχωρίσιμες.

In [38]:
from sklearn.svm import SVC
svmclass = SVC(kernel='rbf')
svmclass.fit(sentences2google_train, corpus[1])
print(svmclass.score(sentences2google_test,corpus_test[1]))

/home/zerkes/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8069


Για το δικό μας μοντέλο

In [39]:
svmclass2 = SVC(kernel='rbf')
svmclass2.fit(sentences2vec_train, corpus[1])
print(svmclass2.score(sentences2vec_test,corpus_test[1]))

/home/zerkes/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.655


Από ότι φαίνεται τα μοντέλα SVM , KNN είναι χειρότερες επιλογές από το logistic regression. Επίσης, είναι εμφανές ότι πάνω στο δικό μας word2vec είναι ακόμα χειρότερα τα ποσοστά, καθώς έχουμε πολύ μικρότερο όγκο δεδομένων από τη google.

__(b)__

__fastText__

In [40]:
!pip install fasttext

     |████████████████████████████████| 61kB 519kB/s eta 0:00:01
  Stored in directory: /home/zerkes/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


### Διαφορές fasttext με word2vec

Το μοντέλο της google (word2vec) αντιμετωπίζει την κάθε λέξη σαν μια οντότητα. Σε αντίθεση το fasttext αντιμετωπίζει την κάθε λέξη σαν σύνθεση χαρακτήρων n-grams. Για παράδειγμα για τη λέξη "text" έχουμε το άθροισμα των διανυσμάτων των n-grams π.χ. "te" ,"tex" , "text" ,"xt", "ext"...

Αυτό έχει ως αποτέλεσμα να έχουμε καλύτερα word embeddings για σπάνιες λέξεις, καθώς στο word2vec μια σπάνια λέξη έχει λίγους γείτονες ώστοσο τα n-grams της μοιράζοντε με αρκετές άλλες λέξεις.

Ωστόσο το fasttext είναι πολύ πιο αργό στο training από το word2vec και ειδικά για μεγάλα dataset.

In [42]:
from gensim.models import FastText

θα κάνω train το fasttext στο corpus τoυ βιβλίου που κατεβάσαμε. Και έπειτα θα δω πόσο ακριβή similarities έχω στις 10 λέξεις που τσεκάραμε και πόσο καλό accuracy έχει στο prediction των imdb reviews.

In [57]:
fast_model = model = FastText(size=20, window=5, min_count=1) 
corpus_sentence_tkn = corpus_sentence_tokenize(path_sherlock)

In [58]:
fast_model.build_vocab(sentences=corpus_sentence_tkn)
fast_model.train(sentences=corpus_sentence_tkn, total_examples=fast_model.corpus_count, epochs=1000) 

Χρησιμοποιούμε size=20 λόγω έλλειψης μνήμης.

Tώρα θα δούμε τα similarities για τις 10 λέξεις που είχαμε δει προηγουμένως με το δικό μας μοντέλο στο βιβλίο που διαλέξαμε.

In [59]:
for i in random_words:
    print("Most Similar words for: ",i)
    print(fast_model.most_similar(positive=i))
    print("\n")

Most Similar words for:  vestige


/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('pronounce', 0.8173289895057678), ('revellers', 0.7642024755477905), ('form', 0.7633427381515503), ('irresistible', 0.7490476965904236), ('detained', 0.7369130849838257), ('descent', 0.7355394959449768), ('attained', 0.7329529523849487), ('representative', 0.7312352657318115), ('valet', 0.7256011962890625), ('force', 0.7237144708633423)]


Most Similar words for:  allude
[('alive', 0.7770902514457703), ('forget', 0.6869080066680908), ('lose', 0.6856810450553894), ('stay', 0.6743953824043274), ('neither', 0.6619927883148193), ('exclude', 0.6454653739929199), ('stiff', 0.6434610486030579), ('philadelphia', 0.6414873600006104), ('do', 0.6405911445617676), ('wit', 0.6259635090827942)]


Most Similar words for:  fulfill
[('fulfil', 0.90674889087677), ('wreck', 0.7742279767990112), ('college', 0.7578479647636414), ('michael', 0.7531304359436035), ('foppishness', 0.7388917207717896), ('shepherd', 0.7387361526489258), ('swamp', 0.7336680889129639), ('chat', 0.7333285212516785), ('cotton', 0.

Σε αυτό το κομμάτι θα κάνουμε ένα training-test με logistic regression όπως πριν αλλά με το fasttext model.

H σύγκριση αυτή είναι για να δούμε ποιο από τα 2 μοντέλα μας προσφέρει καλύτερο accuracy.

In [60]:
index_fast = set(fast_model.wv.index2word)
sentences2fast_train=[]
for i in range(len(corpus[0])):
    sentences2fast_train.append(calculate_sentences(corpus[0][i], fast_model, index_fast,size=20))

sentences2fast_test=[]
for i in range(len(corpus_test[0])):
    sentences2fast_test.append(calculate_sentences(corpus_test[0][i], fast_model, index_fast,size=20))

#logistic regression fit
clffast = LogisticRegression(solver='lbfgs',max_iter=1000)
clffast.fit(sentences2fast_train,corpus[1])
#predict
print(clffast.score(sentences2fast_test,corpus_test[1]))

/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


0.642


Παρατηρούμε πως έχουμε ένα ποσοστό 64% , που είναι 5% λιγότερο από του fast model. Αυτό μπορεί να συμβαίνει διότι στο fasttext έχουμε χρησιμοποιήσει size = 20 και όχι 100 όπως στο word2vec λόγω έλλειψης μνήμης.

Μπορούμε να φανταστούμε πως με ένα αρκετά καλύτερο dataset και μεγαλύτερη μνήμη για να το κάνουμε training σε μεγαλύτερο size θα έχουμε αρκετά καλύτερα αποτελέσματα σε σχέση με το word2vec μοντέλο.